<b>Максим, привет!</b>

Меня зовут Евгений Дащенко, и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропусков](#null)
    * [Замена типа данных](#data_type)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemmas)
    * [Категоризация данных](#categorizing)
3. [Ответы на вопросы](#answers)
4. [Общий вывод](#conclusion)
5. [Чек-лист готовности проекта](#check)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться в концепции / деталях работы. 

</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию 
<a id="start"></a>

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод** В датасете есть пропуски в столбцах 'days_employed' и 'total_income', которые нужно будет заполнить. А так же в столбце 'days_employed' существуют отрицательные значения. Их нужно привести в положительные числа.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Первичные данные посмотрели, давай теперь их обработаем.

</div>

## Шаг 2. Предобработка данных
<a id="preprocessing"></a>

### Обработка пропусков
<a id="null"></a>

In [3]:
#приводим отрицательные значения в столбце 'days_employed' в положительные числа
df['days_employed'] = df['days_employed'].abs()

#вычисляем кол-во строк, где оба столбца пропущены
len(df[df['days_employed'].isnull() & df['total_income'].isnull()]) 

#смотрим какое количество пропусков по типу занятости и выводим долю пропусков по типу занятости
df.set_index('income_type').isna().sum(level=0)
df.set_index('income_type').isna().sum(level=0)/df['days_employed'].isna().sum()

#смотрим какое количество пропусков по уровню образования клиента и выводим долю пропусков по уровню образования клиента
df.set_index('education_id').isna().sum(level=0) 
df.set_index('education_id').isna().sum(level=0)/df['days_employed'].isna().sum() 

#смотрим какое количество пропусков по семейному положению и выводим долю пропусков по семейному положению
df.set_index('family_status').isna().sum(level=0) #смотрим какое количество пропусков по семейному положению
df.set_index('family_status').isna().sum(level=0)/df['days_employed'].isna().sum()

#смотрим какое количество пропусков по полу клиента и выводим долю пропусков по полу клиента
df.set_index('gender').isna().sum(level=0) 
df.set_index('gender').isna().sum(level=0)/df['days_employed'].isna().sum() 

# вычисляем медиану в столбце "total_income" для каждого "income_type"
median_income = df.groupby('income_type').agg({'total_income' : 'median'})
# вычисляем медиану в столбце "days_employed" для каждого "income_type"
median_days = df.groupby(['income_type']).agg({'days_employed' : 'median'}) 

# заполнение пропусков в столбце "total_income" на "median_income"
for inc_type in median_income.index:
    df.loc[df['income_type'] == inc_type,'total_income'] = df.loc[df['income_type'] == inc_type,'total_income'].fillna(median_income.loc[inc_type, 'total_income'])
# заполнение пропусков в столбце "days_employed" на "median_days"
for days_type in median_days.index:
    df.loc[df['income_type'] == days_type,'days_employed'] = df.loc[df['income_type'] == days_type,'days_employed'].fillna(median_days.loc[days_type, 'days_employed']) 
#проверка
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличный способ изменить отрицательные значения в абсолютные.

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе метод `fillna` и `astype` можно применять к нескольким столбцам одновременно.
    
Если столбцов более 2–ух, то рекомендую применять цикл `for`.
    
Пример: 
    
`df.astype({ "col1": "float64", "col2": "int64"})`
    
</div>

**Вывод** Так как не видно существенной корреляции пропусков по типу занятости, по уровню образования клиента, по семейному положению и по полу клиента, можно предположить, что пропуски появились из-за человеческого фактора. Например, клиент не захотел заполнять графы со стажем и доходом, или сотрудник посчитал что эта информация не играет роли и не заполнил её в систему. Общее количество пропусков равно в столбцах "days_employed" и "total_income" равно 2174, а также количество строк, где оба столбца имеют пропуски тоже 2174, значит оба этих столбца содержат пропуски одновременно. Половина пропусков у "сотрудников", а типы занятости без пропусков: "безработный", "студент" и "в декрете". 
**Заполнение пропусков:** так как данные по разным типам занятости часто отличаются друг от друг, выводить общую медиану было бы неправильно. Сначала вывели медиану в столбцах "days_employed" и "total_income" по типу занятости. Затем заполнили пропуски соответственной медианой по типу занятости.


### Замена типа данных
<a id="data_type"></a>

In [4]:
#замена типа данных в столбцах "days_employed" и "total_income"
df = df.astype({'days_employed': int, 'total_income': int}) 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Кстати можно выбрать сразу нужные столбцы и поменять в них тип данных.
    
Оставил пример ниже.
    
</div>

In [ ]:
df[["days_employed", "total_income"]] = df[["days_employed", "total_income"]].astype('int')

**Вывод** Тип float не всегда корректно отображает целочисленные значения. Если нет явной нужды в float, то лучше переводить в int. Это позволяет избежать неявных ошибок. Был выбран метод .astype(), он используется, когда мы хотим заменить один тип данных столбца с другим типом данных. Мы также можем использовать этот метод для одновременного изменения нескольких типов столбцов как в нашем примере.

### Обработка дубликатов
<a id="duplicates"></a>

In [5]:
#приводим к одному регистру столбци 'education', 'family_status', 'gender', 'income_type', 'purpose'
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
#проверяем уникальные значения в солбцах
df['children'].unique()
df['gender'].unique()
df['education'].unique()
df['family_status'].unique()
df['income_type'].unique()
#заменяем кол-во детей т.к. скорее всего 20 и -1 опечатки
df['children'] = df['children'].replace(20 , 2).replace(-1 , 1)
#удаляем 'xna' как неправельное значение в столбце 'gender'
df = df[df.gender != 'xna']
#удаляем дубликаты
df = df.drop_duplicates().reset_index(drop=True)
#проверяем наличие дубликатов
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Сперва мы привели к одному регистру столбцы, где есть вероятность что значения будут отличаться только регистром. Затем проверили уникальные значения в этих столбцах. Заменили кол-во детей, где скорее всего произошла опечатка (20 на 2 и -1 на 1) и удалили значение 'xna' из столбца 'gender'. В итоге с помощью метода .drop_duplicates() мы удалили абсолютные дубликаты. Возможные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Хорошо, что произвели текст в нижний регистр, как раз таким образом мы смогли устранить неявные дубликаты.
    
Сами дубликаты  нам ни к чему и очень здорово, что мы их удалили 😊
    
</div>

### Лемматизация
<a id="lemmas"></a>

In [6]:
#получим лемматизатора для слов на русском
from pymystem3 import Mystem
m = Mystem()
#лемматизация строк с русским текстом в столбце 'purpose' и создание столбца с леммами
df['lemmas'] = df['purpose'].apply(m.lemmatize)
#подсчетаем значения в саписке лемм
from collections import Counter
df['lemmas'].apply(lambda x: x.remove('\n'))
lemmas_sum = Counter(df['lemmas'].sum())
#выводим этот список в порядке убывания
lemmas_sum.most_common()

[(' ', 33569),
 ('недвижимость', 6350),
 ('покупка', 5896),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

**Вывод** Сначала получили лемматизатор для слов на русском языке. Лемматизировали строки с русским текстом в столбце 'purpose' и создали столбец с леммами. Подсчетали значения в саписке лемм и вывели этот список в порядке убывания.

### Категоризация данных
<a id="categorizing"></a>

In [8]:
#создадим "словари"
education_dict = df[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

family_dict = df[['family_status_id','family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)

In [9]:
#объеденим данные в категории и создаём категории цели кредита
def some_func(lemmas):
    #леммы 'жилье' и 'недвижимость' объединим в одно значение 'недвижимость'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
            return 'недвижимость'
    if ('автомобиль' in lemmas):
            return 'автомобиль'
    if ('образование' in lemmas):
            return 'образование'
    if ('свадьба' in lemmas):
            return 'свадьба'
#создадим столбец для категорий цели кредита       
df['purpose_category'] = df['lemmas'].apply(some_func)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Категории кредитов опредилили верно 👍
    
</div>

In [10]:
#создаём категории уровня дохода
quantile_frame = df.quantile(q=[0.25, 0.5, 0.75])

def inc_func(total_income):
    if (total_income <= quantile_frame.loc[0.25]['total_income']):
        return 'низкий'
    if (total_income > quantile_frame.loc[0.25]['total_income']) and (total_income <= quantile_frame.loc[0.5]['total_income']):
        return 'средний'
    if (total_income > quantile_frame.loc[0.5]['total_income']) and (total_income <= quantile_frame.loc[0.75]['total_income']):
        return 'высокий'
    if (total_income > quantile_frame.loc[0.75]['total_income']):
        return 'сверхвысокий'
#создадим столбец для категорий уровня дохода
df['income_category'] = df['total_income'].apply(inc_func)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличный способ категоризовать данные по уровню доходов 👍
    
</div>

In [11]:
#создаём категории наличия детей
def kid_func(children):
    if (children <= 0):
        return 0
    if (children > 0):
        return 1
#создадим столбец для категорий наличия детей
df['kid_category'] = df['children'].apply(kid_func)
print(df.head())

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      f   сотрудник     0        253875   
1   женат / замужем                 0      f   сотрудник     0        112080   
2   женат / замужем                 0      m   сотрудник     0        145885   
3   женат / замужем                 0      m   сотрудник     0        267628   
4  гражданский брак                 1      f   пенсионер     0        158616   

                      purpose                            lemmas  \
0               покупка жилья               [покупка,  , жиль

**Вывод** Были выделены два словаря: education_dict и family_dict т. к. к каждому уровню образования ‘education’ и семейного положения ‘family_status’ есть соответствующий ‘id’. 
Также, были созданы категории **цели кредита**, **уровня дохода** и **наличия детей** с помощью которых мы ответим на вопросы о зависимости в Шаге 3.

## Шаг 3. Ответы на вопросы
<a id="answers"></a>

- Зависимость между **наличием детей** и возвратом кредита в срок

In [15]:
#создадим сводную таблицу 
pvt_child = df.pivot_table(index = 'kid_category', columns = 'debt', values = 'income_type', aggfunc = 'count')
#подщитаем долю должников к общиму количеству заёмщиков
pvt_child['share'] = pvt_child[1]/(pvt_child[0] + pvt_child[1])
#выводим сводную тадлицу на экран
print(pvt_child)

debt              0     1     share
kid_category                       
0             13027  1063  0.075444
1              6685   678  0.092082


**Вывод** По результату исследования видим, что заёмщики, у которых **нет детей**, имеют на 1,7% **меньшую вероятность** не выплатить кредит в срок. 

- Зависимость между **семейным положением** и возвратом кредита в срок

In [14]:
#создадим сводную таблицу
pvt_family = df.pivot_table(index = 'family_status', columns = 'debt', values = 'income_type', aggfunc = 'count')
#подщитаем долю должников к общиму количеству заёмщиков
pvt_family['share'] = pvt_family[1]/(pvt_family[0] + pvt_family[1])
#выводим сводную тадлицу на экран
print(pvt_family)

debt                       0    1     share
family_status                              
в разводе               1110   85  0.071130
вдовец / вдова           896   63  0.065693
гражданский брак        3762  388  0.093494
женат / замужем        11408  931  0.075452
не женат / не замужем   2536  274  0.097509


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В целом логика расчета верная 👍
    
</div>

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b>

    
Это все мможно посчитать с помощью сводных таблиц `pivot_table`, хочу напомнить, что сводные таблицы это те же датафремы. То есть в них можно добавлять столбцы и делать нужные нам расчеты.
    
Давай поправим пожалуйста этот момент.
    
</div>

**Вывод** Анализ показал, что 9,8% заёмщиков в категории "не женат / не замужем" не выплатили кредит в срок. А из тех кто в категории "вдовец / вдова", всего 6,6% просрочтили кредит. Но данные не совершенствованы, т. к. на пример один и тот же заёмщик может однрвременно состоять в более чем одной категории семейного положения.

- Зависимость между **уровнем дохода** и возвратом кредита в срок

In [16]:
#создадим сводную таблицу
pvt_income = df.pivot_table(index = 'income_category', columns = 'debt', values = 'income_type', aggfunc = 'count')
#подщитаем долю должников к общиму количеству заёмщиков
pvt_income['share'] = pvt_income[1]/(pvt_income[0] + pvt_income[1])
#выводим сводную тадлицу на экран
print(pvt_income)

debt                0    1     share
income_category                     
высокий          4799  448  0.085382
низкий           4937  427  0.079605
сверхвысокий     4980  383  0.071415
средний          4996  483  0.088155


**Вывод** Возвратом кредита в срок зависит меньше всего от **уровня дохода**. Но мы можем отметить, что лишь 7,1% заёмщиков в категории дохода "сверхвысокий" т. е. с ежемесячным доходом более **195818**, не вернули кредит в срок. У заёмщиков в трёх остольных категориях дохода приблизительно одинаковый процент невозврата кредита в срок.

- Влияние **цели кредита** на его возврат в срок

In [17]:
#создадим сводную таблицу
pvt_purpose = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'income_type', aggfunc = 'count')
#подщитаем долю должников к общиму количеству заёмщиков
pvt_purpose['share'] = pvt_purpose[1]/(pvt_purpose[0] + pvt_purpose[1])
#выводим сводную тадлицу на экран
print(pvt_purpose)

debt                  0    1     share
purpose_category                      
автомобиль         3903  403  0.093590
недвижимость      10028  782  0.072340
образование        3643  370  0.092200
свадьба            2138  186  0.080034


**Вывод** Самый высокий процент невозврата кредита в срок был замечан у заёмщиков с **целью кредита** "автомобиль" и "образование", 9,4% и 9,2% соответсвенно. С другой стороны, всего лишь 7,2% заёмщиков с целью кредита "недвижимость" не вернули кредит в срок.

## Шаг 4. Общий вывод
<a id="conclusion"></a>

Для того чтобы разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, было сначала необходимо привести данные к подходящему виду. После провеждения анализа, мы смогли определить какие категории заёмщиков более склонны просрочиванию выплаты кредита. Банку стоит отметить, что теоритически заёмщики могут одновременно относиться более чем к одной категории семейного положения, что может привести к неправильному результату в исследовании. В общем исследование показало что, клиент **с детьми**, **вне брака**, у которого ежемесячный доход **не более 195818** и у которого цель кредита **"автомобиль"** имеет портрет самого **ненадёжного** заёмщика.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
    
Достаточно детально проделанная работа и это не может не радовать 😊
    
Давай поправим мои комментарии и будем двигаться дальше.
    
Если у тебя вдруг есчть какие–либо вопросы, то я с радостью отвечу на них 😊
       

</div>

## Чек-лист готовности проекта
<a id="check"></a>


Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [X]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.